In [5]:
# data load

In [2]:
import json
import pandas as pd

file_paths = [
    "/data/jaesung/llm_for_diabetes/src/model/unsloth/model_output/classification_model_output.jsonl",
    "/data/jaesung/llm_for_diabetes/src/model/unsloth/model_output/generation_model_output_2.jsonl"
]

data = []
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))

df = pd.DataFrame(data)

In [8]:
# medqa

import pandas as pd
qa1 = df[df['task']=='qa1']

correct_pred = (qa1['output'].str.lower() == qa1['model_output_128'].str.lower()).sum()
total_pred = len(qa1)

acc = correct_pred / total_pred
print(acc)

0.57


In [9]:
# medmcqa

import pandas as pd

qa2 = df[df['task']=='qa2']

correct_pred = (qa2['output'].str.lower() == qa2['model_output_128'].str.lower()).sum()
total_pred = len(qa2)

acc = correct_pred / total_pred
print(acc)

0.66


In [10]:
# pubmedqa

import pandas as pd

qa3 = df[df['task']=='qa3']

correct_pred = (qa3['output'].str.lower() == qa3['model_output_128'].str.lower()).sum()
total_pred = len(qa3)

acc = correct_pred / total_pred
print(acc)

0.96


In [11]:
# nli

import pandas as pd

nli = df[df['task']=='nli']

correct_pred = (nli['output'].str.lower() == nli['model_output_128'].str.lower()).sum()
total_pred = len(nli)

acc = correct_pred / total_pred
print(acc)

0.5657894736842105


In [13]:
# re

import pandas as pd

re = df[df['task'] == 're']

def calculate_total_metrics(output_col, model_output_col):
    total_true_positive = 0
    total_false_positive = 0
    total_false_negative = 0

    for output, model_output in zip(output_col, model_output_col):
        # ','로 구분된 문자열을 집합으로 변환
        output_set = set(output.split(', '))
        model_output_set = set(model_output.split(', '))

        # 교집합, 정답의 크기, 모델 예측의 크기 계산
        true_positive = len(output_set & model_output_set)
        false_positive = len(model_output_set - output_set)
        false_negative = len(output_set - model_output_set)

        # 누적 합산
        total_true_positive += true_positive
        total_false_positive += false_positive
        total_false_negative += false_negative

    # 총 Precision, Recall, F1 계산
    precision = total_true_positive / (total_true_positive + total_false_positive) if (total_true_positive + total_false_positive) > 0 else 0
    recall = total_true_positive / (total_true_positive + total_false_negative) if (total_true_positive + total_false_negative) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# 메트릭 계산
total_precision, total_recall, total_f1 = calculate_total_metrics(re['output'], re['model_output_128'])

# 결과 출력
print(f"Total Precision: {total_precision:.4f}")
print(f"Total Recall: {total_recall:.4f}")
print(f"Total F1-Score: {total_f1:.4f}")


Total Precision: 0.8500
Total Recall: 0.8500
Total F1-Score: 0.8500


In [15]:
# ie

from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# 데이터프레임 예시 (df_ie 가 주어진 데이터프레임)
ie = df[df['task'] == 'ie']

ie["output"] = ie["output"].str.lower().str.split(", ")
ie["model_output_128"] = ie["model_output_128"].str.lower().str.split(", ")

# Precision, Recall, F1-score 계산 함수
def calculate_scores(y_true, y_pred):
    all_precisions = []
    all_recalls = []
    all_f1s = []
    
    for true_vals, pred_vals in zip(y_true, y_pred):
        true_set = set(true_vals) if isinstance(true_vals, list) else set()
        pred_set = set(pred_vals) if isinstance(pred_vals, list) else set()

        TP = len(true_set & pred_set)  # True Positives (정답과 예측이 일치하는 것)
        FP = len(pred_set - true_set)  # False Positives (예측했지만 정답이 아닌 것)
        FN = len(true_set - pred_set)  # False Negatives (정답이지만 예측하지 못한 것)

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1s.append(f1)

    return sum(all_precisions) / len(all_precisions), sum(all_recalls) / len(all_recalls), sum(all_f1s) / len(all_f1s)

# Precision, Recall, F1-score 계산
precision, recall, f1 = calculate_scores(ie["output"], ie["model_output_128"])

# 결과 출력
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Precision: 0.6861
Recall: 0.5680
F1-score: 0.5840


/tmp/ipykernel_567074/368763351.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ie["output"] = ie["output"].str.lower().str.split(", ")
/tmp/ipykernel_567074/368763351.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ie["model_output_128"] = ie["model_output_128"].str.lower().str.split(", ")


In [4]:
# summarization

import openai
import pandas as pd
import re
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# BLEURT 모델 로드
bleurt_model_name = "Elron/bleurt-large-512"
tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model = bleurt_model.to(device)

MAX_CONTEXT_LENGTH = 8192

# GPT-4 평가 함수
def evaluate_with_gpt4(input_text, model_output, true_output):
    input_text = str(input_text)[:MAX_CONTEXT_LENGTH]
    model_output = str(model_output)[:MAX_CONTEXT_LENGTH]
    true_output = str(true_output)[:MAX_CONTEXT_LENGTH]

    prompt = f"""
    You are tasked with evaluating the quality of a summarization model's responses based on the following metrics:

    1. **Coherence**: Does the model's response logically align with the context provided in the input and the true answer? Does it stay on topic without unnecessary deviation?  
    - **Deduct points if the response contains hallucinated or factually incorrect information.**
    
    2. **Completeness**: Does the model's response sufficiently and accurately answer the question in the input? Is any critical information missing?  
    - **Deduct points if hallucinations cause important details to be misleading or incorrect.**
    
    3. **Naturalness**: Does the model's response sound fluent, natural, and human-like?  
    - **Deduct points if hallucinations make the response sound unnatural or unrealistic.**

    ⚠️ **Penalty for Hallucination**:  
    - If the model generates information that is factually incorrect, fabricated, or not supported by the input and true answer, **reduce scores for Coherence, Completeness, and Naturalness accordingly.**  
    - Severe hallucinations should result in significantly lower scores.

    **Input**:
    {input_text}

    **Model's Response**:
    {model_output}

    **True Answer**:
    {true_output}

    Please rate each metric on a scale from 1 to 5, factoring in hallucinations as a major penalty.

    Example response format:
    - Coherence: X.X
    - Completeness: X.X
    - Naturalness: X.X
    """

    try:
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for Summarization models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

# GPT-4 점수 추출
def extract_scores(evaluation):
    if evaluation is None:
        return {"Coherence": 0.0, "Completeness": 0.0, "Naturalness": 0.0}
    coherence = re.search(r"Coherence: (\d\.\d)", evaluation)
    completeness = re.search(r"Completeness: (\d\.\d)", evaluation)
    naturalness = re.search(r"Naturalness: (\d\.\d)", evaluation)
    
    return {
        "Coherence": float(coherence.group(1)) if coherence else 0.0,
        "Completeness": float(completeness.group(1)) if completeness else 0.0,
        "Naturalness": float(naturalness.group(1)) if naturalness else 0.0
    }

# BLEURT 점수 계산
def calculate_bleurt(y_true, y_pred):
    inputs = tokenizer(y_pred, y_true, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        scores = bleurt_model(**inputs).logits

    if scores.numel() == 1:
        return float(scores.squeeze().item())  
    return [float(score) for score in scores.squeeze().tolist()]  

# BLEURT 및 BERTScore 계산
def calculate_bleurt_and_bertscore(y_true, y_pred):
    bleurt_score_value = calculate_bleurt(y_true, y_pred)
    _, _, bert_f1 = bert_score(y_pred, y_true, lang="en", rescale_with_baseline=True)
    bert_f1_avg = sum(bert_f1) / len(bert_f1) if len(bert_f1) > 0 else 0

    return {
        "BLEURT": bleurt_score_value if isinstance(bleurt_score_value, float) else sum(bleurt_score_value) / len(bleurt_score_value),
        "BERTScore_F1": bert_f1_avg
    }

# 점수 정규화 함수
def normalize_scores(df, column):
    if column not in df.columns:
        print(f"Warning: Column {column} not found in DataFrame. Skipping normalization.")
        return df
    df[column] = df[column].apply(lambda x: float(x) if isinstance(x, torch.Tensor) else x)
    min_val, max_val = df[column].min(), df[column].max()
    df[column] = df[column].apply(lambda x: (x - min_val) / (max_val - min_val) if max_val > min_val else 0.5)
    return df

summarization = df[df['task'] == 'summarization']
results = []

for _, row in summarization.iterrows():
    input_text = str(row['input'])
    model_output = str(row['model_output_2048'])
    true_output = str(row['output'])
    
    evaluation = evaluate_with_gpt4(input_text, model_output, true_output)
    print(f"Evaluation result:\n{evaluation}")
    
    scores = extract_scores(evaluation)
    metric_scores = calculate_bleurt_and_bertscore([true_output], [model_output])

    results.append({
        "input": input_text,
        "model_output": model_output,
        "true_output": true_output,
        "evaluation": evaluation,
        "Coherence": scores["Coherence"],
        "Completeness": scores["Completeness"],
        "Naturalness": scores["Naturalness"],
        "BLEURT": metric_scores.get("BLEURT", 0.0),  # 기본값 설정
        "BERTScore_F1": metric_scores.get("BERTScore_F1", 0.0)
    })

evaluation_df = pd.DataFrame(results)

# 'BLEURT' 컬럼이 존재하는지 확인 후 정규화 수행
evaluation_df = normalize_scores(evaluation_df, "BLEURT")
evaluation_df = normalize_scores(evaluation_df, "BERTScore_F1")

average_scores = evaluation_df[["Coherence", "Completeness", "Naturalness", "BLEURT", "BERTScore_F1"]].mean()
print("평균 점수:")
print(average_scores)


Evaluation result:
Based on the evaluation criteria provided and the comparison between the Model's Response and the True Answer, here are the ratings for each metric:

- **Coherence: 2.0**
  - The model's response lacks coherence as it introduces significant hallucinated information such as the patient's age, medical history differences, dates, and additional symptoms that were not present in the input or the true answer. This deviation from the original context results in a lack of alignment with the provided information.

- **Completeness: 2.5**
  - The model's response partially captures some key elements like the initial symptoms and some test results. However, the inclusion of fabricated details and the omission of crucial information result in an incomplete portrayal of the patient's medical history and condition. The hallucinated information negatively impacts the accuracy and completeness of the response.

- **Naturalness: 2.0**
  - The model's response sounds somewhat unnatur

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
Based on the evaluation metrics provided and considering the presence of hallucinations as a major penalty, here is the rating for the model's response:

- Coherence: 2.0
- Completeness: 2.5
- Naturalness: 3.0

Explanation:

1. **Coherence**: The model's response contains some relevant information but also introduces significant changes and inaccuracies compared to the true answer. Key details such as the age of the patient, the amount of purulent material obtained during the fine-needle aspiration, and the physical examination findings are incorrect or exaggerated. The response lacks alignment with the context and introduces hallucinated information.
   
2. **Completeness**: The model partially addresses the symptoms, medical history, diagnostic procedures, and treatment plan mentioned in the input but fails to accurately represent crucial details. Significant deviations from the true answer, incorrect measurements, and fabricated information reduce the completeness

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
Based on the provided information, I would evaluate the model's response as follows:

- Coherence: 2.5
- Completeness: 2.0
- Naturalness: 2.5

Explanation:
1. **Coherence**:
   - The model's response contains some hallucinated information and deviates from the true answer, particularly in the patient's age and a few details about the medical history. The response does follow the general context but introduces inaccuracies.
   
2. **Completeness**:
   - The model fails to provide a complete and accurate answer in terms of the patient's age, medication details, and treatment outcomes, resulting in critical omissions and erroneous information.
   
3. **Naturalness**:
   - The response maintains a somewhat fluent and natural tone, but the hallucinations and inaccuracies detract from its overall naturalness.

Overall, the model's response demonstrates significant hallucinations and inaccuracies, impacting its coherence, completeness, and naturalness. Further refinement is

...

Evaluation result:
- Coherence: 3.0
- Completeness: 2.5
- Naturalness: 3.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation result:
- Coherence: 3.0
- Completeness: 3.5
- Naturalness: 3.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


평균 점수:
Coherence       2.881579
Completeness    2.828947
Naturalness     2.947368
BLEURT          0.794822
BERTScore_F1    0.610959
dtype: float64


In [ ]:
# generation

import openai
import pandas as pd
import re
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# BLEURT 모델 로드
bleurt_model_name = "Elron/bleurt-large-512"
tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model = bleurt_model.to(device)

MAX_CONTEXT_LENGTH = 8192

# GPT-4 평가 함수
def evaluate_with_gpt4(input_text, model_output, true_output):
    input_text = str(input_text)[:MAX_CONTEXT_LENGTH]
    model_output = str(model_output)[:MAX_CONTEXT_LENGTH]
    true_output = str(true_output)[:MAX_CONTEXT_LENGTH]

    prompt = f"""
    You are tasked with evaluating the quality of a QA model's responses based on the following metrics:
    1. **Coherence**: Does the model's response logically align with the context provided in the input?
    2. **Completeness**: Does the model's response sufficiently answer the question in the input?
    3. **Naturalness**: Does the model's response sound fluent and human-like?

    **Input**:
    {input_text}

    **Model's Response**:
    {model_output}

    **True Answer**:
    {true_output}

    Please rate each metric on a scale from 1 to 5. 
    Example response format:
    - Coherence: X.X
    - Completeness: X.X
    - Naturalness: X.X
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for Summarization models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

# GPT-4 점수 추출
def extract_scores(evaluation):
    if evaluation is None:
        return {"Coherence": 0.0, "Completeness": 0.0, "Naturalness": 0.0}
    coherence = re.search(r"Coherence: (\d\.\d)", evaluation)
    completeness = re.search(r"Completeness: (\d\.\d)", evaluation)
    naturalness = re.search(r"Naturalness: (\d\.\d)", evaluation)
    
    return {
        "Coherence": float(coherence.group(1)) if coherence else 0.0,
        "Completeness": float(completeness.group(1)) if completeness else 0.0,
        "Naturalness": float(naturalness.group(1)) if naturalness else 0.0
    }

# BLEURT 점수 계산
def calculate_bleurt(y_true, y_pred):
    inputs = tokenizer(y_pred, y_true, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        scores = bleurt_model(**inputs).logits

    if scores.numel() == 1:
        return float(scores.squeeze().item())  
    return [float(score) for score in scores.squeeze().tolist()]  

# BLEURT 및 BERTScore 계산
def calculate_bleurt_and_bertscore(y_true, y_pred):
    bleurt_score_value = calculate_bleurt(y_true, y_pred)
    _, _, bert_f1 = bert_score(y_pred, y_true, lang="en", rescale_with_baseline=True)
    bert_f1_avg = sum(bert_f1) / len(bert_f1) if len(bert_f1) > 0 else 0

    return {
        "BLEURT": bleurt_score_value if isinstance(bleurt_score_value, float) else sum(bleurt_score_value) / len(bleurt_score_value),
        "BERTScore_F1": bert_f1_avg
    }

# 점수 정규화 함수
def normalize_scores(df, column):
    if column not in df.columns:
        print(f"Warning: Column {column} not found in DataFrame. Skipping normalization.")
        return df
    df[column] = df[column].apply(lambda x: float(x) if isinstance(x, torch.Tensor) else x)
    min_val, max_val = df[column].min(), df[column].max()
    df[column] = df[column].apply(lambda x: (x - min_val) / (max_val - min_val) if max_val > min_val else 0.5)
    return df

generation = df[df['task'] == 'generation']
results = []

for _, row in generation.iterrows():
    input_text = str(row['input'])
    model_output = str(row['model_output_512'])
    true_output = str(row['output'])
    
    evaluation = evaluate_with_gpt4(input_text, model_output, true_output)
    print(f"Evaluation result:\n{evaluation}")
    
    scores = extract_scores(evaluation)
    metric_scores = calculate_bleurt_and_bertscore([true_output], [model_output])

    results.append({
        "input": input_text,
        "model_output": model_output,
        "true_output": true_output,
        "evaluation": evaluation,
        "Coherence": scores["Coherence"],
        "Completeness": scores["Completeness"],
        "Naturalness": scores["Naturalness"],
        "BLEURT": metric_scores.get("BLEURT", 0.0),  # 기본값 설정
        "BERTScore_F1": metric_scores.get("BERTScore_F1", 0.0)
    })

evaluation_df = pd.DataFrame(results)

# 'BLEURT' 컬럼이 존재하는지 확인 후 정규화 수행
evaluation_df = normalize_scores(evaluation_df, "BLEURT")
evaluation_df = normalize_scores(evaluation_df, "BERTScore_F1")

average_scores = evaluation_df[["Coherence", "Completeness", "Naturalness", "BLEURT", "BERTScore_F1"]].mean()
print("평균 점수:")
print(average_scores)


KeyError: "None of [Index(['Coherence', 'Completeness', 'Naturalness', 'BLEURT', 'BERTScore_F1'], dtype='object')] are in the [columns]"

: 

In [27]:
# daily_diets

import openai
import pandas as pd
import re
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# BLEURT 모델 로드
bleurt_model_name = "Elron/bleurt-large-512"
tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model = bleurt_model.to(device)

MAX_CONTEXT_LENGTH = 8192

# GPT-4 평가 함수
def evaluate_with_gpt4(input_text, model_output, true_output):
    input_text = str(input_text)[:MAX_CONTEXT_LENGTH]
    model_output = str(model_output)[:MAX_CONTEXT_LENGTH]
    true_output = str(true_output)[:MAX_CONTEXT_LENGTH]

    prompt = f"""
    You are tasked with evaluating the quality of a QA model's responses based on the following metrics:
    1. **Coherence**: Does the model's response logically align with the context provided in the input?
    2. **Completeness**: Does the model's response sufficiently answer the question in the input?
    3. **Naturalness**: Does the model's response sound fluent and human-like?

    **Input**:
    {input_text}

    **Model's Response**:
    {model_output}

    **True Answer**:
    {true_output}

    Please rate each metric on a scale from 1 to 5. 
    Example response format:
    - Coherence: X.X
    - Completeness: X.X
    - Naturalness: X.X
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for Summarization models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

# GPT-4 점수 추출
def extract_scores(evaluation):
    if evaluation is None:
        return {"Coherence": 0.0, "Completeness": 0.0, "Naturalness": 0.0}
    coherence = re.search(r"Coherence: (\d\.\d)", evaluation)
    completeness = re.search(r"Completeness: (\d\.\d)", evaluation)
    naturalness = re.search(r"Naturalness: (\d\.\d)", evaluation)
    
    return {
        "Coherence": float(coherence.group(1)) if coherence else 0.0,
        "Completeness": float(completeness.group(1)) if completeness else 0.0,
        "Naturalness": float(naturalness.group(1)) if naturalness else 0.0
    }

# BLEURT 점수 계산
def calculate_bleurt(y_true, y_pred):
    inputs = tokenizer(y_pred, y_true, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        scores = bleurt_model(**inputs).logits

    if scores.numel() == 1:
        return float(scores.squeeze().item())  
    return [float(score) for score in scores.squeeze().tolist()]  

# BLEURT 및 BERTScore 계산
def calculate_bleurt_and_bertscore(y_true, y_pred):
    bleurt_score_value = calculate_bleurt(y_true, y_pred)
    _, _, bert_f1 = bert_score(y_pred, y_true, lang="en", rescale_with_baseline=True)
    bert_f1_avg = sum(bert_f1) / len(bert_f1) if len(bert_f1) > 0 else 0

    return {
        "BLEURT": bleurt_score_value if isinstance(bleurt_score_value, float) else sum(bleurt_score_value) / len(bleurt_score_value),
        "BERTScore_F1": bert_f1_avg
    }

# 점수 정규화 함수
def normalize_scores(df, column):
    if column not in df.columns:
        print(f"Warning: Column {column} not found in DataFrame. Skipping normalization.")
        return df
    df[column] = df[column].apply(lambda x: float(x) if isinstance(x, torch.Tensor) else x)
    min_val, max_val = df[column].min(), df[column].max()
    df[column] = df[column].apply(lambda x: (x - min_val) / (max_val - min_val) if max_val > min_val else 0.5)
    return df

daily = df[df['task'] == 'daily_diets']
results = []

for i, row in daily.iterrows():
    input_text = str(row['input'])
    model_output = str(row['model_output_512'])
    true_output = str(row['output'])
    
    evaluation = evaluate_with_gpt4(input_text, model_output, true_output)
    if i % 10 == 0:
        print(f"Evaluation {i+1} result:\n{evaluation}")
    
    scores = extract_scores(evaluation)
    metric_scores = calculate_bleurt_and_bertscore([true_output], [model_output])

    results.append({
        "input": input_text,
        "model_output": model_output,
        "true_output": true_output,
        "evaluation": evaluation,
        "Coherence": scores["Coherence"],
        "Completeness": scores["Completeness"],
        "Naturalness": scores["Naturalness"],
        "BLEURT": metric_scores.get("BLEURT", 0.0),  # 기본값 설정
        "BERTScore_F1": metric_scores.get("BERTScore_F1", 0.0)
    })

evaluation_df = pd.DataFrame(results)

# 'BLEURT' 컬럼이 존재하는지 확인 후 정규화 수행
evaluation_df = normalize_scores(evaluation_df, "BLEURT")
evaluation_df = normalize_scores(evaluation_df, "BERTScore_F1")

average_scores = evaluation_df[["Coherence", "Completeness", "Naturalness", "BLEURT", "BERTScore_F1"]].mean()
print("평균 점수:")
print(average_scores)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Evaluation 671 result:
- Coherence: 4.0
- Completeness: 4.5
- Naturalness: 4.0

Overall, the model's response aligns well with the context provided, but there are some differences in the meal plan details compared to the true answer. The response is quite complete in addressing the requirements of the input, but there is room for improvement in making it sound more fluent and natural.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Evaluation 681 result:
- Coherence: 4.0
- Completeness: 4.5
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

...

Evaluation 921 result:
- Coherence: 4.0
- Completeness: 3.0
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Evaluation 931 result:
- Coherence: 4.0
- Completeness: 4.5
- Naturalness: 4.0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

평균 점수:
Coherence       3.905109
Completeness    3.538321
Naturalness     3.742701
BLEURT          0.601100
BERTScore_F1    0.649711
dtype: float64


In [28]:
# daily_diets

import openai
import pandas as pd
import re
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# BLEURT 모델 로드
bleurt_model_name = "Elron/bleurt-large-512"
tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)
bleurt_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bleurt_model = bleurt_model.to(device)

MAX_CONTEXT_LENGTH = 8192

# GPT-4 평가 함수
def evaluate_with_gpt4(input_text, model_output, true_output):
    input_text = str(input_text)[:MAX_CONTEXT_LENGTH]
    model_output = str(model_output)[:MAX_CONTEXT_LENGTH]
    true_output = str(true_output)[:MAX_CONTEXT_LENGTH]

    prompt = f"""
    You are tasked with evaluating the quality of a QA model's responses based on the following metrics:
    1. **Coherence**: Does the model's response logically align with the context provided in the input?
    2. **Completeness**: Does the model's response sufficiently answer the question in the input?
    3. **Naturalness**: Does the model's response sound fluent and human-like?

    **Input**:
    {input_text}

    **Model's Response**:
    {model_output}

    **True Answer**:
    {true_output}

    Please rate each metric on a scale from 1 to 5. 
    Example response format:
    - Coherence: X.X
    - Completeness: X.X
    - Naturalness: X.X
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "You are an expert evaluator for Summarization models."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error with GPT-4 API:", e)
        return None

# GPT-4 점수 추출
def extract_scores(evaluation):
    if evaluation is None:
        return {"Coherence": 0.0, "Completeness": 0.0, "Naturalness": 0.0}
    coherence = re.search(r"Coherence: (\d\.\d)", evaluation)
    completeness = re.search(r"Completeness: (\d\.\d)", evaluation)
    naturalness = re.search(r"Naturalness: (\d\.\d)", evaluation)
    
    return {
        "Coherence": float(coherence.group(1)) if coherence else 0.0,
        "Completeness": float(completeness.group(1)) if completeness else 0.0,
        "Naturalness": float(naturalness.group(1)) if naturalness else 0.0
    }

# BLEURT 점수 계산
def calculate_bleurt(y_true, y_pred):
    inputs = tokenizer(y_pred, y_true, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        scores = bleurt_model(**inputs).logits

    if scores.numel() == 1:
        return float(scores.squeeze().item())  
    return [float(score) for score in scores.squeeze().tolist()]  

# BLEURT 및 BERTScore 계산
def calculate_bleurt_and_bertscore(y_true, y_pred):
    bleurt_score_value = calculate_bleurt(y_true, y_pred)
    _, _, bert_f1 = bert_score(y_pred, y_true, lang="en", rescale_with_baseline=True)
    bert_f1_avg = sum(bert_f1) / len(bert_f1) if len(bert_f1) > 0 else 0

    return {
        "BLEURT": bleurt_score_value if isinstance(bleurt_score_value, float) else sum(bleurt_score_value) / len(bleurt_score_value),
        "BERTScore_F1": bert_f1_avg
    }

# 점수 정규화 함수
def normalize_scores(df, column):
    if column not in df.columns:
        print(f"Warning: Column {column} not found in DataFrame. Skipping normalization.")
        return df
    df[column] = df[column].apply(lambda x: float(x) if isinstance(x, torch.Tensor) else x)
    min_val, max_val = df[column].min(), df[column].max()
    df[column] = df[column].apply(lambda x: (x - min_val) / (max_val - min_val) if max_val > min_val else 0.5)
    return df

alternative = df[df['task'] == 'alternative_diet']
results = []

for _, row in alternative.iterrows():
    input_text = str(row['input'])
    model_output = str(row['model_output_512'])
    true_output = str(row['output'])
    
    evaluation = evaluate_with_gpt4(input_text, model_output, true_output)
    if i % 10 == 0:
        print(f"Evaluation {i+1} result:\n{evaluation}")
            
    scores = extract_scores(evaluation)
    metric_scores = calculate_bleurt_and_bertscore([true_output], [model_output])

    results.append({
        "input": input_text,
        "model_output": model_output,
        "true_output": true_output,
        "evaluation": evaluation,
        "Coherence": scores["Coherence"],
        "Completeness": scores["Completeness"],
        "Naturalness": scores["Naturalness"],
        "BLEURT": metric_scores.get("BLEURT", 0.0),  # 기본값 설정
        "BERTScore_F1": metric_scores.get("BERTScore_F1", 0.0)
    })

evaluation_df = pd.DataFrame(results)

# 'BLEURT' 컬럼이 존재하는지 확인 후 정규화 수행
evaluation_df = normalize_scores(evaluation_df, "BLEURT")
evaluation_df = normalize_scores(evaluation_df, "BERTScore_F1")

average_scores = evaluation_df[["Coherence", "Completeness", "Naturalness", "BLEURT", "BERTScore_F1"]].mean()
print("평균 점수:")
print(average_scores)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

평균 점수:
Coherence       3.487805
Completeness    3.250000
Naturalness     3.425610
BLEURT          0.538604
BERTScore_F1    0.562781
dtype: float64


In [29]:
import pandas as pd

dfh = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/diabetes_food_hub_new_nutri_facts.csv")
dfh.head(2)

,title,description,prep_time,cook_time,servings,steps,tags,nutrition_facts,ingredients
0,Raspberry Swirl Frozen Yogurt Bark,Raspberry Swirl Frozen Yogurt Bark: Dive into ...,10 min,4 hr,6 Servings,['Cover a freezer-safe tray with parchment pap...,"['Kid Friendly', 'Vegetarian', 'Dessert', 'Sna...","{'Servings': '6 Servings', 'Serving Size': '1 ...","[{'label': 'Plain Nonfat Greek yogurt', 'us_me..."
1,Maple-Pumpkin Spice Oatmeal Cookies,Description not found,10 min,25 min,14 Servings,['Preheat the oven to 350 degrees F. Line two ...,"['Kid Friendly', 'Vegetarian', 'Snacks', 'Glut...","{'Servings': '14 Servings', 'Serving Size': '1...","[{'label': 'old-fashioned rolled oats', 'us_me..."


In [31]:
# daily diet - nutri score

import ast
import pandas as pd
import json
import re
import openai
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from dotenv import load_dotenv
import os
from tqdm import tqdm

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_numeric_value(value):
    try:
        if isinstance(value, str):
            match = re.search(r"(\d+(\.\d+)?)", value)
            if match:
                return float(match.group(1))
        elif isinstance(value, (int, float)):
            return float(value)
    except Exception as e:
        print(f"Error in extract_numeric_value: {e}, value: {value}")
    return 0.0

def is_valid_meal_structure(json_string):
    try:
        data = json.loads(json_string)
        return all(key in data for key in ['Breakfast', 'Lunch', 'Dinner'])
    except (json.JSONDecodeError, TypeError):
        return False

def find_most_similar_row(title, dfh):
    try:
        dfh['title'] = dfh['title'].fillna('')  # Handle NaN values
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(dfh['title'])
        input_vector = vectorizer.transform([title])
        similarities = cosine_similarity(input_vector, tfidf_matrix)
        most_similar_idx = similarities.argmax()
        return dfh.iloc[most_similar_idx]
    except Exception as e:
        print(f"Error in find_most_similar_row: {e}, title: {title}")
        return None

def identify_fruit_veg(ingredients_list):
    try:
        prompt = f"Identify which items in the following ingredient list are fruits or vegetables:\n\n{ingredients_list}\n\nReturn only the names of items that are fruits or vegetables in a Python list format."
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant identifying fruits and vegetables."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0
        )
        fruits_vegetables = response['choices'][0]['message']['content']
        return ast.literal_eval(fruits_vegetables)
    except Exception as e:
        print(f"Error identifying fruits and vegetables: {e}")
        return []

def calculate_fruit_veg_points(ingredients, total_weight):
    try:
        ingredients_list = ast.literal_eval(ingredients)
        fruit_veg_labels = identify_fruit_veg(ingredients_list)

        fruit_veg_weight = 0
        for ingredient in ingredients_list:
            label = ingredient.get('label', '')
            weight = extract_numeric_value(ingredient.get('metric_measure', 0))
            if label in fruit_veg_labels:
                fruit_veg_weight += weight

        # 과일/채소 비율을 100g 기준으로 변환
        fruit_veg_ratio = (fruit_veg_weight / total_weight) * 100 if total_weight > 0 else 0

        if fruit_veg_ratio > 80:
            return 5
        elif fruit_veg_ratio > 60:
            return 2
        elif fruit_veg_ratio > 40:
            return 1
        else:
            return 0
    except Exception as e:
        print(f"Error calculating fruit_veg_points: {e}")
        return 0

def extract_nested_value(data, keys, default=0):
    try:
        for key in keys:
            if isinstance(data, dict):
                data = data.get(key, {})
            else:
                return default
        return extract_numeric_value(data) if isinstance(data, (int, float, str)) else default
    except Exception as e:
        print(f"Error in extract_nested_value: {e}, keys: {keys}, data: {data}")
        return default

def calculate_nutri_score(nutrition_facts, ingredients):
    try:
        if isinstance(nutrition_facts, str):
            nutrition_facts = ast.literal_eval(nutrition_facts)

        # 전체 무게 계산
        total_weight = sum(
            extract_numeric_value(ingredient.get('metric_measure', 0)) 
            for ingredient in ast.literal_eval(ingredients)
        )
        if total_weight == 0:
            print("Warning: Total weight is zero. Skipping calculation.")
            return None

        # 100g 기준으로 성분 정규화
        energy = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Calories']) / total_weight * 100
        saturated_fat = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Fat', 'Amount']) / total_weight * 100
        sugar = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Total Sugars']) / total_weight * 100
        sodium = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Sodium']) / total_weight * 100
        fiber = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Dietary Fiber']) / total_weight * 100
        protein = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Protein']) / total_weight * 100

        # Unfavorable points calculation
        energy_points = min(energy / 80, 800)
        saturated_fat_points = min(saturated_fat / 1, 10)
        sugar_points = min(sugar / 4.5, 45)
        sodium_points = min(sodium / 90, 900)

        unfavorable_points = energy_points + saturated_fat_points + sugar_points + sodium_points

        # Favorable points calculation
        fiber_points = min(fiber / 0.7, 3.5)
        protein_points = min(protein / 1.6, 8.0)
        fruit_veg_points = calculate_fruit_veg_points(ingredients, total_weight)

        favorable_points = fiber_points + protein_points + fruit_veg_points

        # Final Nutri-Score calculation
        total_score = unfavorable_points - favorable_points
        return total_score
    except Exception as e:
        print(f"Error in calculate_nutri_score: {e}, nutrition_facts: {nutrition_facts}")
        return None

def get_nutri_score_grade(score):
    if score <= -1:
        return "A"
    elif score <= 2:
        return "B"
    elif score <= 10:
        return "C"
    elif score <= 18:
        return "D"
    else:
        return "E"

def calculate_meal_nutri_score(meal_data, dfh):
    meal_scores = {}

    for meal, title in meal_data.items():
        matched_row = find_most_similar_row(title, dfh)
        if matched_row is None:
            continue

        nutrition_facts = matched_row['nutrition_facts']
        ingredients = matched_row['ingredients']
        score = calculate_nutri_score(nutrition_facts, ingredients)

        if score is None:
            print(f"Warning: Nutri-Score calculation failed for meal '{meal}' with title '{title}'.")
            grade = "N/A"
        else:
            grade = get_nutri_score_grade(score)

        meal_scores[meal] = {'score': score, 'grade': grade}

    return meal_scores

def calculate_scores_with_comparison(df, dfh):
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        output_scores = {}
        model_scores = {}
        if is_valid_meal_structure(row.get('output', '')):
            output_data = json.loads(row['output'])
            output_scores = calculate_meal_nutri_score(output_data, dfh)
        if is_valid_meal_structure(row.get('model_output_512', '')):
            model_data = json.loads(row['model_output_512'])
            model_scores = calculate_meal_nutri_score(model_data, dfh)
        results.append({'row_index': idx, 'output_scores': output_scores, 'model_scores': model_scores})
    return results

def calculate_average_scores(results):
    """
    Calculate the average Nutri-Scores for outputs and model outputs.
    """
    output_total_score = 0
    model_total_score = 0
    output_count = 0
    model_count = 0

    for result in results:
        # Extract output scores
        for meal, score_data in result['output_scores'].items():
            if score_data['score'] is not None:
                output_total_score += score_data['score']
                output_count += 1

        # Extract model scores
        for meal, score_data in result['model_scores'].items():
            if score_data['score'] is not None:
                model_total_score += score_data['score']
                model_count += 1

    # Calculate averages
    output_avg = output_total_score / output_count if output_count > 0 else None
    model_avg = model_total_score / model_count if model_count > 0 else None

    return output_avg, model_avg


# 'daily_diets' task Nutri-Score calculation
filtered_df = df[df['task'] == 'daily_diets']
results = calculate_scores_with_comparison(filtered_df, dfh)

# Calculate overall averages
output_avg, model_avg = calculate_average_scores(results)

# Print results
print("=== Results for Each Row ===")
for result in results:
    print(f"Row Index: {result['row_index']}")
    print(f"Output Scores: {result['output_scores']}")
    print(f"Model Output Scores: {result['model_scores']}")
    print()

print("=== Overall Averages ===")
print(f"Output Average Nutri-Score: {output_avg}")
print(f"Model Output Average Nutri-Score: {model_avg}")


100%|██████████| 274/274 [00:00<00:00, 22063.42it/s]

=== Results for Each Row ===
Row Index: 661
Output Scores: {}
Model Output Scores: {}

Row Index: 662
Output Scores: {}
Model Output Scores: {}

Row Index: 663
Output Scores: {}
Model Output Scores: {}

Row Index: 664
Output Scores: {}
Model Output Scores: {}

Row Index: 665
Output Scores: {}
Model Output Scores: {}

Row Index: 666
Output Scores: {}
Model Output Scores: {}

Row Index: 667
Output Scores: {}
Model Output Scores: {}

Row Index: 668
Output Scores: {}
Model Output Scores: {}

Row Index: 669
Output Scores: {}
Model Output Scores: {}

Row Index: 670
Output Scores: {}
Model Output Scores: {}

Row Index: 671
Output Scores: {}
Model Output Scores: {}

Row Index: 672
Output Scores: {}
Model Output Scores: {}

Row Index: 673
Output Scores: {}
Model Output Scores: {}

Row Index: 674
Output Scores: {}
Model Output Scores: {}

Row Index: 675
Output Scores: {}
Model Output Scores: {}

Row Index: 676
Output Scores: {}
Model Output Scores: {}

Row Index: 677
Output Scores: {}
Model Outp

In [32]:
# alternative diet - nutri score

import ast
import pandas as pd
import json
import re
import openai
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from dotenv import load_dotenv
import os
from tqdm import tqdm

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_numeric_value(value):
    try:
        if isinstance(value, str):
            match = re.search(r"(\d+(\.\d+)?)", value)
            if match:
                return float(match.group(1))
        elif isinstance(value, (int, float)):
            return float(value)
    except Exception as e:
        print(f"Error in extract_numeric_value: {e}, value: {value}")
    return 0.0

def is_valid_meal_structure(json_string):
    try:
        data = json.loads(json_string)
        return isinstance(data, dict)
    except (json.JSONDecodeError, TypeError):
        return False

def find_most_similar_row(title, dfh):
    try:
        dfh['title'] = dfh['title'].fillna('')  # Handle NaN values
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(dfh['title'])
        input_vector = vectorizer.transform([title])
        similarities = cosine_similarity(input_vector, tfidf_matrix)
        most_similar_idx = similarities.argmax()
        return dfh.iloc[most_similar_idx]
    except Exception as e:
        print(f"Error in find_most_similar_row: {e}, title: {title}")
        return None

def identify_fruit_veg(ingredients_list):
    try:
        prompt = f"Identify which items in the following ingredient list are fruits or vegetables:\n\n{ingredients_list}\n\nReturn only the names of items that are fruits or vegetables in a Python list format."
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant identifying fruits and vegetables."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0
        )
        fruits_vegetables = response['choices'][0]['message']['content']
        return ast.literal_eval(fruits_vegetables)
    except Exception as e:
        print(f"Error identifying fruits and vegetables: {e}")
        return []

def calculate_fruit_veg_points(ingredients, total_weight):
    try:
        ingredients_list = ast.literal_eval(ingredients)
        fruit_veg_labels = identify_fruit_veg(ingredients_list)

        fruit_veg_weight = 0
        for ingredient in ingredients_list:
            label = ingredient.get('label', '')
            weight = extract_numeric_value(ingredient.get('metric_measure', 0))
            if label in fruit_veg_labels:
                fruit_veg_weight += weight

        # 과일/채소 비율을 100g 기준으로 변환
        fruit_veg_ratio = (fruit_veg_weight / total_weight) * 100 if total_weight > 0 else 0

        if fruit_veg_ratio > 80:
            return 5
        elif fruit_veg_ratio > 60:
            return 2
        elif fruit_veg_ratio > 40:
            return 1
        else:
            return 0
    except Exception as e:
        print(f"Error calculating fruit_veg_points: {e}")
        return 0

def extract_nested_value(data, keys, default=0):
    try:
        for key in keys:
            if isinstance(data, dict):
                data = data.get(key, {})
            else:
                return default
        return extract_numeric_value(data)
    except Exception as e:
        print(f"Error in extract_nested_value: {e}, keys: {keys}, data: {data}")
        return default

def calculate_nutri_score(nutrition_facts, ingredients):
    try:
        if isinstance(nutrition_facts, str):
            nutrition_facts = ast.literal_eval(nutrition_facts)

        # 전체 무게 계산
        total_weight = sum(
            extract_numeric_value(ingredient.get('metric_measure', 0)) 
            for ingredient in ast.literal_eval(ingredients)
        )
        if total_weight == 0:
            print("Warning: Total weight is zero. Skipping calculation.")
            return None

        # 100g 기준으로 성분 정규화
        energy = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Calories']) / total_weight * 100
        saturated_fat = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Fat', 'Amount']) / total_weight * 100
        sugar = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Total Sugars']) / total_weight * 100
        sodium = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Sodium']) / total_weight * 100
        fiber = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Total Carbohydrates', 'Dietary Fiber']) / total_weight * 100
        protein = extract_nested_value(nutrition_facts, ['Amount per Serving', 'Protein']) / total_weight * 100

        # Unfavorable points calculation
        energy_points = min(energy / 80, 800)
        saturated_fat_points = min(saturated_fat / 1, 10)
        sugar_points = min(sugar / 4.5, 45)
        sodium_points = min(sodium / 90, 900)

        unfavorable_points = energy_points + saturated_fat_points + sugar_points + sodium_points

        # Favorable points calculation
        fiber_points = min(fiber / 0.7, 3.5)
        protein_points = min(protein / 1.6, 8.0)
        fruit_veg_points = calculate_fruit_veg_points(ingredients, total_weight)

        favorable_points = fiber_points + protein_points + fruit_veg_points

        # Final Nutri-Score calculation
        total_score = unfavorable_points - favorable_points
        return total_score
    except Exception as e:
        print(f"Error in calculate_nutri_score: {e}, nutrition_facts: {nutrition_facts}")
        return None

def get_nutri_score_grade(score):
    if score <= -1:
        return "A"
    elif score <= 2:
        return "B"
    elif score <= 10:
        return "C"
    elif score <= 18:
        return "D"
    else:
        return "E"

def calculate_scores_with_comparison_no_meals(df, dfh):
    results = []
    output_scores_list = []
    model_output_scores_list = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            output_text = row.get('output', '')
            if output_text:
                matched_row = find_most_similar_row(output_text, dfh)
                if matched_row is not None:
                    nutrition_facts = matched_row['nutrition_facts']
                    ingredients = matched_row['ingredients']
                    output_score = calculate_nutri_score(nutrition_facts, ingredients)
                    output_scores_list.append(output_score)
                else:
                    output_score = None

            model_output_text = row.get('model_output_512', '')
            if model_output_text:
                matched_row = find_most_similar_row(model_output_text, dfh)
                if matched_row is not None:
                    nutrition_facts = matched_row['nutrition_facts']
                    ingredients = matched_row['ingredients']
                    model_output_score = calculate_nutri_score(nutrition_facts, ingredients)
                    model_output_scores_list.append(model_output_score)
                else:
                    model_output_score = None

            results.append({
                'row_index': idx,
                'output_score': output_score,
                'model_output_score': model_output_score
            })

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            results.append({
                'row_index': idx,
                'output_score': None,
                'model_output_score': None
            })

    final_output_avg = sum(output_scores_list) / len(output_scores_list) if output_scores_list else None
    final_model_output_avg = sum(model_output_scores_list) / len(model_output_scores_list) if model_output_scores_list else None

    print(f"Output Average Nutri-Score: {final_output_avg}")
    print(f"Model Output Average Nutri-Score: {final_model_output_avg}")

    return results

# Execution
filtered_df = df[df['task'] == 'alternative_diet']
results = calculate_scores_with_comparison_no_meals(filtered_df, dfh)

# Print results
for result in results:
    print(result)


  0%|          | 0/246 [00:00<?, ?it/s]

 64%|██████▍   | 158/246 [03:34<01:46,  1.21s/it]

Error identifying fruits and vegetables: unterminated string literal (detected at line 1) (<unknown>, line 1)


 84%|████████▍ | 207/246 [04:37<00:40,  1.03s/it]

100%|██████████| 246/246 [05:27<00:00,  1.33s/it]


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'